In [63]:
import re
import time
import MeCab
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from collections import Counter
from collections import OrderedDict
from gensim.models import keyedvectors
from gensim.models import word2vec
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

In [2]:
All_df = pd.read_pickle("data/all_mecab.pickle")
All_df

,sen,sen_pre,say_id,reply_id,group_id,name,body,16types_a,16types_b,argument_a,argument_b,epistemic_a,epistemic_b,social_a,social_b,coordination_a,coordination_b
0,"[よろしく, お願い, し, ます, ！, EOS]",[EOS],1,-1,1234568,まこぴす,よろしくお願いします！,5,5,1,1,1,1,0,0,0,0
1,"[よろしく, お願い, し, ます, EOS]","[よろしく, お願い, し, ます, ！, EOS]",31,-1,1234568,哲,よろしくお願いします,5,5,1,1,1,1,0,0,0,0
2,"[名前, な, の, が, 恥ずかしい, です, EOS, よろしく, お願い, し, ます...","[よろしく, お願い, し, ます, EOS]",70,-1,1234568,仙波,名前なのが恥ずかしいです…\nよろしくお願いします！,5,5,1,1,1,1,0,0,0,0
3,"[早速, 課題, やっ, ちゃ, い, ましょ, う, ！, EOS]","[名前, な, の, が, 恥ずかしい, です, EOS, よろしく, お願い, し, ます...",119,-1,1234568,まこぴす,早速課題やっちゃいましょう！,14,14,1,1,1,1,0,0,2,2
4,"[やっ, ちゃ, い, ましょ, う, EOS, ｍｏｏｄｌｅ, は, ゴミ, です, ！,...","[早速, 課題, やっ, ちゃ, い, ましょ, う, ！, EOS]",163,-1,1234568,仙波,やっちゃいましょう\nmoodleはゴミです！,1,2,2,2,2,2,1,0,0,0
5,"[使い, にくい, です, 笑, EOS]","[やっ, ちゃ, い, ましょ, う, EOS, ｍｏｏｄｌｅ, は, ゴミ, です, ！,...",194,-1,1234568,まこぴす,使いにくいです(笑),2,2,2,2,2,2,1,0,0,0
6,"[同意, です, EOS]","[使い, にくい, です, 笑, EOS]",302,-1,1234568,哲,同意です,1,1,2,2,2,2,2,2,0,0
7,"[以前, インタラクティブ, アート, 受講, し, て, い, た, の, です, が, ...","[同意, です, EOS]",309,-1,1234568,仙波,以前インタラクティブアート受講していたのですが、その時に課題が不具合で出せなくなっていた時期...,2,2,3,3,2,2,1,1,0,0
8,"[学習, 環境, として, 必要, 最低限, の, 機能, は, 備え, て, いる, と,...","[以前, インタラクティブ, アート, 受講, し, て, い, た, の, です, が, ...",385,-1,1234568,まこぴす,学習環境として必要最低限の機能は備えていると思うが、操作性の面ではPCの最低限の知識があるこ...,2,2,5,5,2,2,1,1,0,0
9,"[自分, の, 意見, です, ！, 笑, EOS]","[学習, 環境, として, 必要, 最低限, の, 機能, は, 備え, て, いる, と,...",426,-1,1234568,まこぴす,自分の意見です！(笑),2,2,1,2,1,2,0,1,0,0


In [3]:
sen = All_df['sen'].values
sen_pre = All_df['sen_pre'].values

# 単語辞書の作成
wd_set = Counter([x for s in (sen + sen_pre) for x in s])
wd_ary = np.array(list(wd_set.keys()))
wd_cnt = np.array(list(wd_set.values()))

# 出現頻度順にソート
wd_ary = wd_ary[np.argsort(wd_cnt)[::-1]]
wd_cnt.sort()
wd_cnt = wd_cnt[::-1]

# 単語ID辞書の作成
wd_to_id = {wd: i for i, wd in enumerate(wd_ary)}

# Top10の単語を出力
for i in range(10):
    print("単語:",
          list(wd_ary)[i], "\t出現数:",
          list(wd_cnt)[i], "\tID:", wd_to_id[list(wd_ary)[i]])

# 出現数CUT_OFF以下の単語のIDを統一
CUT_OFF = 2
print("words kinds:", len(wd_cnt), "words>=" + str(CUT_OFF) + ":",
      np.sum(wd_cnt >= CUT_OFF))
other_id = np.sum(wd_cnt >= CUT_OFF)
wd_to_id.update({wd: other_id for wd in wd_ary[wd_cnt < CUT_OFF]})
id_to_wd = {wd_to_id[wd]: wd for wd in wd_to_id.keys()}

# senとsen_preの単語をIDに変換
sen_id = []
sen_pre_id = []
for s, s_pre in zip(sen, sen_pre):
    sen_id.append([str(wd_to_id[wd]) for wd in s])
    sen_pre_id.append([str(wd_to_id[wd]) for wd in s_pre])

# 新し列としてAll_dfに追加
All_df.insert(loc=0, column='sen_id', value=sen_id)
All_df.insert(loc=1, column='sen_pre_id', value=sen_pre_id)

単語: EOS 	出現数: 29580 	ID: 0
単語: の 	出現数: 10567 	ID: 1
単語: て 	出現数: 7408 	ID: 2
単語: です 	出現数: 7390 	ID: 3
単語: ます 	出現数: 7363 	ID: 4
単語: か 	出現数: 7285 	ID: 5
単語: 、 	出現数: 6959 	ID: 6
単語: に 	出現数: 6914 	ID: 7
単語: が 	出現数: 6893 	ID: 8
単語: は 	出現数: 6793 	ID: 9
words kinds: 6961 words>=2: 6649


In [4]:
All_df

,sen_id,sen_pre_id,sen,sen_pre,say_id,reply_id,group_id,name,body,16types_a,16types_b,argument_a,argument_b,epistemic_a,epistemic_b,social_a,social_b,coordination_a,coordination_b
0,"[39, 35, 12, 4, 18, 0]",[0],"[よろしく, お願い, し, ます, ！, EOS]",[EOS],1,-1,1234568,まこぴす,よろしくお願いします！,5,5,1,1,1,1,0,0,0,0
1,"[39, 35, 12, 4, 0]","[39, 35, 12, 4, 18, 0]","[よろしく, お願い, し, ます, EOS]","[よろしく, お願い, し, ます, ！, EOS]",31,-1,1234568,哲,よろしくお願いします,5,5,1,1,1,1,0,0,0,0
2,"[415, 19, 1, 8, 2253, 3, 0, 39, 35, 12, 4, 18, 0]","[39, 35, 12, 4, 0]","[名前, な, の, が, 恥ずかしい, です, EOS, よろしく, お願い, し, ます...","[よろしく, お願い, し, ます, EOS]",70,-1,1234568,仙波,名前なのが恥ずかしいです…\nよろしくお願いします！,5,5,1,1,1,1,0,0,0,0
3,"[1797, 45, 142, 302, 56, 54, 27, 18, 0]","[415, 19, 1, 8, 2253, 3, 0, 39, 35, 12, 4, 18, 0]","[早速, 課題, やっ, ちゃ, い, ましょ, う, ！, EOS]","[名前, な, の, が, 恥ずかしい, です, EOS, よろしく, お願い, し, ます...",119,-1,1234568,まこぴす,早速課題やっちゃいましょう！,14,14,1,1,1,1,0,0,2,2
4,"[142, 302, 56, 54, 27, 0, 657, 9, 2246, 3, 18, 0]","[1797, 45, 142, 302, 56, 54, 27, 18, 0]","[やっ, ちゃ, い, ましょ, う, EOS, ｍｏｏｄｌｅ, は, ゴミ, です, ！,...","[早速, 課題, やっ, ちゃ, い, ましょ, う, ！, EOS]",163,-1,1234568,仙波,やっちゃいましょう\nmoodleはゴミです！,1,2,2,2,2,2,1,0,0,0
5,"[196, 919, 3, 112, 0]","[142, 302, 56, 54, 27, 0, 657, 9, 2246, 3, 18, 0]","[使い, にくい, です, 笑, EOS]","[やっ, ちゃ, い, ましょ, う, EOS, ｍｏｏｄｌｅ, は, ゴミ, です, ！,...",194,-1,1234568,まこぴす,使いにくいです(笑),2,2,2,2,2,2,1,0,0,0
6,"[1794, 3, 0]","[196, 919, 3, 112, 0]","[同意, です, EOS]","[使い, にくい, です, 笑, EOS]",302,-1,1234568,哲,同意です,1,1,2,2,2,2,2,2,0,0
7,"[1619, 1406, 4329, 746, 12, 2, 56, 10, 1, 3, 8...","[1794, 3, 0]","[以前, インタラクティブ, アート, 受講, し, て, い, た, の, です, が, ...","[同意, です, EOS]",309,-1,1234568,仙波,以前インタラクティブアート受講していたのですが、その時に課題が不具合で出せなくなっていた時期...,2,2,3,3,2,2,1,1,0,0
8,"[44, 424, 118, 168, 1796, 1, 181, 9, 4215, 2, ...","[1619, 1406, 4329, 746, 12, 2, 56, 10, 1, 3, 8...","[学習, 環境, として, 必要, 最低限, の, 機能, は, 備え, て, いる, と,...","[以前, インタラクティブ, アート, 受講, し, て, い, た, の, です, が, ...",385,-1,1234568,まこぴす,学習環境として必要最低限の機能は備えていると思うが、操作性の面ではPCの最低限の知識があるこ...,2,2,5,5,2,2,1,1,0,0
9,"[68, 1, 132, 3, 18, 112, 0]","[44, 424, 118, 168, 1796, 1, 181, 9, 4215, 2, ...","[自分, の, 意見, です, ！, 笑, EOS]","[学習, 環境, として, 必要, 最低限, の, 機能, は, 備え, て, いる, と,...",426,-1,1234568,まこぴす,自分の意見です！(笑),2,2,1,2,1,2,0,1,0,0


In [72]:
sen_id = All_df['sen_id'].values
sen_pre_id = All_df['sen_pre_id'].values
sen_all = np.hstack((sen_id, sen_pre_id))

max_sen_length = max([len(sen) for sen in sen_all])
print("max_sen_length:", max_sen_length)

word_vectors_size = 200

# model = word2vec.Word2Vec(
#     sen_all, size=word_vectors_size, min_count=1, window=5)

max_sen_length: 292


In [73]:
def get_w2v_model():
    return word2vec.Word2Vec.load("data/w2v_model")
def train_w2v_model(sents, wv_size):
    model = word2vec.Word2Vec(sents, size=wv_size, min_count=1, window=5)
    model.save("data/w2v_model")

In [74]:
train_w2v_model(sen_all,200)

In [79]:
model = get_w2v_model()
model.wv[str('123')]

array([ 6.7286259e-01,  4.1349065e-01, -3.7735382e-01, -3.6090946e-01,
       -8.2029931e-02, -5.9348220e-01, -7.4297208e-01, -3.8421431e-01,
        2.1897338e-01, -6.4843446e-01, -1.6992661e-01, -4.6462208e-01,
       -2.5559038e-01,  7.5286096e-01,  2.8145101e-02,  3.5276234e-01,
       -9.3863502e-02, -3.2745278e-01,  8.4431261e-01,  1.2577643e-01,
        5.9525728e-01,  4.4892743e-01, -5.7631195e-01,  2.1380819e-01,
        5.0109822e-01, -3.2120851e-01, -9.2305589e-01, -6.0070437e-01,
        3.5558927e-01, -1.2726293e+00,  2.0875604e-01,  4.3196237e-01,
       -1.7237005e-01,  3.1999114e-01,  4.0722492e-01, -9.0266615e-01,
       -8.4801513e-01, -5.9238869e-01,  2.1158533e-01, -1.9347306e-01,
       -8.3067054e-01, -8.3300292e-01, -6.6071606e-01, -1.0786929e-01,
       -4.1340363e-01, -2.1429259e-01,  2.2633922e-01, -4.6710700e-01,
       -8.8519864e-02, -1.2807629e-01, -3.5270309e-01, -3.2630572e-01,
       -4.6155730e-01, -2.5138030e-02, -4.5825124e-01, -1.3587511e-01,
      

In [65]:
model = keyedvectors.KeyedVectors.load_word2vec_format('/Users/kinten/Downloads/entity_vector/entity_vector.model.bin', binary=True)

In [59]:
model.wv['の']

/Users/kinten/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


array([-9.91739094e-01,  4.33873266e-01, -1.63432705e+00, -2.63711929e+00,
        1.87340975e+00, -6.82320058e-01,  1.40024066e+00,  2.55516553e+00,
       -2.15050650e+00, -9.30107951e-01, -4.91664827e-01, -9.65239882e-01,
       -2.25033021e+00, -4.79446650e-01,  1.27358663e+00, -2.34894287e-02,
        1.38359773e+00,  8.10718000e-01, -2.87942588e-01,  3.57353568e-01,
        9.63935137e-01,  1.51740813e+00,  1.57587755e+00,  2.39189360e-02,
        1.13024759e+00,  4.49263763e+00,  9.72327352e-01,  7.27772951e-01,
       -1.67395318e+00,  1.35944712e+00,  1.03268695e+00,  2.55958462e+00,
       -1.54859936e+00,  1.29370260e+00,  1.36028302e+00,  2.94609338e-01,
       -3.44111419e+00, -2.51860529e-01,  1.01693809e+00, -1.33447099e+00,
        1.07817149e+00, -6.26900911e-01, -1.83873641e+00, -1.26155031e+00,
       -1.91633224e-01,  1.44910109e+00,  1.50189626e+00,  3.07009745e+00,
       -1.37203526e+00,  6.46075964e-01, -9.43559185e-02, -1.01390874e+00,
       -1.77591056e-01,  

In [60]:
print(type(wd_ary))
print(wd_ary.shape)
print(wd_ary[0])
print(wd_ary[10])
print(wd_ary[100])

<class 'numpy.ndarray'>
(6961,)
EOS
た
グループ


In [57]:
import mojimoji

# print(model.wv["東京工科"])

sum = 0
sum_no = 0
for i,w in enumerate(wd_ary):
    if w in model.vocab:
        sum+=1
    else:
#         print(i,w)
#         sum_no +=1
        1+2
        if mojimoji.zen_to_han(str(w)) in model.vocab:
            print("aaa",i,w)
            2+3
        else:
#             print("aaa",i,w)
            sum_no +=1
print(sum)
print(sum_no)
print(wd_ary[177])
print(wd_ary[378])
print(mojimoji.zen_to_han(str(wd_ary[378])))

aaa 177 ＬＭＳ
aaa 209 ＳＣＯＲＭ
aaa 225 ＬＲＳ
aaa 283 Ｃａｎ
aaa 291 Ｔｉｎ
aaa 372 ＡＰＩ
aaa 495 ＭＯＯＣ
aaa 567 ＤＳ
aaa 1011 Ｅｘｐｅｒｉｅｎｃｅ
aaa 1070 ＤＯＮ
aaa 1122 Ｒｅｆｅｒｅｎｃｅ
aaa 1140 Ｃｏｎｔｅｎｔ
aaa 1147 Ｍｏｄｅｌ
aaa 1148 Ｏｂｊｅｃｔ
aaa 1245 ｃａｎ
aaa 1321 ｉＴｕｎｅｓ
aaa 1390 ｔｗｉｔｔｅｒ
aaa 1468 ｔｉｎ
aaa 1480 ＵＲＬ
aaa 1553 Ｓｃｒａｔｃｈ
aaa 1653 ＭＩＴ
aaa 1656 ＯＣＷ
aaa 1674 ＹＯ
aaa 1696 ｏｒｚ
aaa 1741 ｍｉｓｓ
aaa 1928 Ｏｈ
aaa 1958 ＬＩＮＥ
aaa 2167 Ｐｏｒｔａｌ
aaa 2169 Ｔｗｉｔｔｅｒ
aaa 2242 ｘＡＰＩ
aaa 2290 ｈｔｍｌ
aaa 2308 Ｅｎｇｌｉｓｈ
aaa 2326 ＴＲＰＧ
aaa 2393 ｙｏｕｔｕｂｅ
aaa 2394 ｉｎｇ
aaa 2476 ａｔ
aaa 2496 Ａｃａｄｅｍｙ
aaa 2511 Ｋｈａｎ
aaa 2542 Ｏｐｅｎ
aaa 2599 ＱＲ
aaa 2623 ＶＰＮ
aaa 2626 ｇｏｏｇｌｅ
aaa 2655 ｏｒ
aaa 2698 ＶＲ
aaa 2720 ｍａｔｃｈ
aaa 2736 ｏｋ
aaa 2737 ｔｅｓｔ
aaa 2819 ＧＯ
aaa 2830 Ｗｏｒｄ
aaa 2863 Ｌｅａｒｎｉｎｇ
aaa 2887 ＳＵ
aaa 2903 ｄａｔａ
aaa 2983 ｅｄｘ
aaa 3156 ＴＯＫＩＯ
aaa 3183 ＢＣＥ
aaa 3220 ａｒｅ
aaa 3227 Ｓｔｕｄｙ
aaa 3247 ｍｉｘｉ
aaa 3305 ｏｎ
aaa 3455 ＤＬ
aaa 3479 ｏｌｄ
aaa 3480 ｒｉｖｅｒ
aaa 3507 ＺＥＲＯ
aaa 3525 ｔｈｉｓ
aaa 3572 Ｓｙｓｔｅｍ
aaa 3593 Ｍａｎａｇｅｍｅｎｔ
aaa 3613 Ｖｅｒ
aaa 3683 Ｙｅｓ
aaa 3703 ＭＪ
aaa 3722 ＳＭＡＰ
aaa 3772 ＯＴＬ
aaa